In [4]:
import sys
sys.path.append("../")
import glob
from hphc.helpers import ProjUNETPrediction, Integer_to_border, MeasureArea, DistWater
from tifffile import imread
from skimage.measure import label
%gui qt

In [ ]:
Masterdir = '/media/sancere/Newton_Volume_1/DalmiroAnalysis/HairData/Avg/'
Resultsdir = '/media/sancere/Newton_Volume_1/DalmiroAnalysis/HairData/Results/'
BinaryName = 'BinaryWater'
HairSegName = 'Water'
VeinImageName = 'Vein'
MaskName = 'Mask'
CsvName = 'Area_Stats'


First we redo the watershed segmentation by adding/removing hair cells

In [ ]:
for fname in filesRaw:
    
    image = imread(fname)
    Name = os.path.basename(os.path.splitext(fname)[0])
    MaskImage = imread(Resultsdir + Name + MaskName + '.tif')
    IntegerImage = imread(Resultsdir + Name + HairSegName + '.tif')
    Veinimage = imread(Resultsdir + Name + VeinImageName + '.tif')
    
    NewMarkerImage = np.zeros(IntegerImage.shape)
    waterproperties = measure.regionprops(IntegerImage)
    Coordinates = [prop.centroid for prop in waterproperties]
    Coordinates = sorted(Coordinates , key=lambda k: [k[0], k[1]]
    viewer.add_points(data = Coordinates, name= 'Markers', face_color='red', ndim = 2)
    
                         
    viewer.show(block=True)
            
    NewCoordinates = viewer.layers['Markers'].data  
    NewMarkerImage[tuple(coordinates_int.T)] = 1 + np.arange(len(NewCoordinates))

    markers = morphology.dilation(NewMarkerImage, morphology.disk(2))  
     
    #Redo Watershed                     
    WaterImage, BinaryImage = AfterUNET(IntegerImage > 0, NewCoordinates, Maskimage, Veinimage) 
                         
    imwrite((Resultsdir  +   Name + MarkerName '.tif' ) , markers) 
    imwrite((Resultsdir  +   Name + HairSegName '.tif' ) , ModifiedArraySeg) 
    imwrite((Resultsdir  +   Name + BinaryName '.tif' ) , BinaryImage)                     

Correct for the segmentation Masks and obtian size statistics

In [ ]:
Raw_path = os.path.join(Masterdir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort

Results_path = os.path.join(Resultsdir, '*tif')
filesResults = glob.glob(Results_path)
filesResults.sor
for fname in filesRaw:
    
    image = imread(fname)
    Name = os.path.basename(os.path.splitext(fname)[0])
    BinaryImage = imread(Resultsdir + Name + BinaryName + '.tif')
    IntegerImage = imread(Resultsdir + Name + HairSegName + '.tif')
   
    MaskImage = imread(Resultsdir + Name + MaskName + '.tif')
    viewer = napari.Viewer()
    viewer.add_image(image)
    viewer.add_labels(IntegerImage, name = 'Integer_Labels')

    viewer.add_labels(BinaryImage, name = 'Binary_Segmentation')
    viewer.add_labels(MaskImage, name = 'Wing_Mask')
    
    #Block viewer
        
    viewer.show(block=True)
            
    ModifiedArraySeg = viewer.layers['Integer_Labels'].data 
    ModifiedArraySeg = ModifiedArraySeg.astype('uint16')
    LabelMaskImage = ModifiedArraySeg > 0
    Compartment = label(LabelMaskImage)
    ModifiedArrayMask = viewer.layers['Wing_Mask'].data 
    ModifiedArrayMask = ModifiedArrayMask.astype('uint8')
    
    BinaryImage = Integer_to_border(ModifiedArraySeg)
    
    imwrite((Resultsdir  +   Name + HairSegName '.tif' ) , ModifiedArraySeg) 
    imwrite((Resultsdir  +   Name + BinaryName '.tif' ) , BinaryImage)
    imwrite((Resultsdir  +   Name + MaskName '.tif' ) , MaskImage)
    MeasureArea(Compartment,LabelMaskImage, Resultsdir, Name)
    